In [1]:
import numpy as np 
import pandas as pd
import json 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score

from tqdm import tqdm
tqdm.pandas()
from sklearn.linear_model import ElasticNet
import statsmodels.api as sm
from copy import deepcopy

import Utils
import models

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel('PMT_march24.xlsx', sheet_name='teens')

In [3]:
df

,age,code,ACC_ADD1,ACC_ADD2,ACC_ADD3,ACC_DIV1,ACC_DIV2,ACC_DIV3,ACC_MUL1,ACC_MUL2,...,N_ADD3,N_DIV1,N_DIV2,N_DIV3,N_MUL1,N_MUL2,N_MUL3,N_SUB1,N_SUB2,N_SUB3
0,15,P-AYU-15-F-9R-ORLO-MOON2021-IDXX-2292,1.000000,0.894737,0.900000,0.840000,0.444444,0.181818,1.000000,0.666667,...,9.0,21.0,4.0,2.0,29.0,6.0,8.0,27.0,9.0,4.0
1,13,P-DKO-13-M-7E-1793-MOON2021-SGXX-2303,0.900000,0.916667,0.714286,0.816327,0.714286,0.625000,0.963636,0.764706,...,10.0,40.0,10.0,5.0,53.0,13.0,6.0,50.0,15.0,5.0
2,13,P-AAD-13-M-7C-1874-MOON2021-SGXX-2323,0.878788,0.750000,0.375000,0.782609,0.500000,0.125000,0.916667,0.350000,...,3.0,18.0,2.0,1.0,22.0,7.0,6.0,22.0,7.0,4.0
3,10,P-ABO-10-F-5I-1514-MOON2021-AFXX-2370,1.000000,0.500000,0.100000,0.789474,0.263158,0.117647,0.800000,0.222222,...,1.0,15.0,5.0,2.0,12.0,2.0,4.0,9.0,5.0,2.0
4,15,P-ABY-15-F-9G-RELI-IZON2021-AIXX-2325,0.914286,0.900000,0.714286,0.815789,0.769231,0.600000,0.921053,0.384615,...,10.0,31.0,10.0,6.0,35.0,5.0,6.0,39.0,10.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,12,P-YPA-12-F-6E-1205-MOON2021-ADXX-2258,0.944444,1.000000,0.750000,0.956522,0.666667,0.666667,0.950000,0.800000,...,3.0,22.0,4.0,2.0,19.0,4.0,0.0,20.0,3.0,5.0
292,17,P-YPO-17-F-BF1-HSEL-MOON2021-APXX-2306,0.936170,0.826087,0.461538,0.944444,0.750000,0.444444,1.000000,0.722222,...,6.0,51.0,9.0,4.0,58.0,13.0,7.0,49.0,15.0,7.0
293,15,P-YVE-15-F-9A-0029-IZON2021-SBXX-2260,0.961538,0.863636,0.631579,0.984375,0.533333,0.800000,0.986111,0.900000,...,12.0,63.0,8.0,8.0,71.0,18.0,4.0,63.0,17.0,9.0
294,14,P-ZBR-14-F-8B-1164-MOON2021-ADXX-2312,0.954545,0.714286,0.666667,0.900000,0.666667,0.857143,0.937500,0.571429,...,6.0,27.0,6.0,6.0,30.0,4.0,2.0,27.0,6.0,3.0


In [4]:
df.columns

Index(['age', 'code', 'ACC_ADD1', 'ACC_ADD2', 'ACC_ADD3', 'ACC_DIV1',
       'ACC_DIV2', 'ACC_DIV3', 'ACC_MUL1', 'ACC_MUL2', 'ACC_MUL3', 'ACC_SUB1',
       'ACC_SUB2', 'ACC_SUB3', 'RT_ADD1', 'RT_ADD2', 'RT_ADD3', 'RT_DIV1',
       'RT_DIV2', 'RT_DIV3', 'RT_MUL1', 'RT_MUL2', 'RT_MUL3', 'RT_SUB1',
       'RT_SUB2', 'RT_SUB3', 'N_ADD1', 'N_ADD2', 'N_ADD3', 'N_DIV1', 'N_DIV2',
       'N_DIV3', 'N_MUL1', 'N_MUL2', 'N_MUL3', 'N_SUB1', 'N_SUB2', 'N_SUB3'],
      dtype='object')

In [5]:


df = df.drop(columns=['code'])

seed = 0xAB0BA
np.random.seed(seed)

In [6]:
df

,age,ACC_ADD1,ACC_ADD2,ACC_ADD3,ACC_DIV1,ACC_DIV2,ACC_DIV3,ACC_MUL1,ACC_MUL2,ACC_MUL3,...,N_ADD3,N_DIV1,N_DIV2,N_DIV3,N_MUL1,N_MUL2,N_MUL3,N_SUB1,N_SUB2,N_SUB3
0,15,1.000000,0.894737,0.900000,0.840000,0.444444,0.181818,1.000000,0.666667,0.347826,...,9.0,21.0,4.0,2.0,29.0,6.0,8.0,27.0,9.0,4.0
1,13,0.900000,0.916667,0.714286,0.816327,0.714286,0.625000,0.963636,0.764706,0.500000,...,10.0,40.0,10.0,5.0,53.0,13.0,6.0,50.0,15.0,5.0
2,13,0.878788,0.750000,0.375000,0.782609,0.500000,0.125000,0.916667,0.350000,0.461538,...,3.0,18.0,2.0,1.0,22.0,7.0,6.0,22.0,7.0,4.0
3,10,1.000000,0.500000,0.100000,0.789474,0.263158,0.117647,0.800000,0.222222,0.235294,...,1.0,15.0,5.0,2.0,12.0,2.0,4.0,9.0,5.0,2.0
4,15,0.914286,0.900000,0.714286,0.815789,0.769231,0.600000,0.921053,0.384615,0.545455,...,10.0,31.0,10.0,6.0,35.0,5.0,6.0,39.0,10.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,12,0.944444,1.000000,0.750000,0.956522,0.666667,0.666667,0.950000,0.800000,0.000000,...,3.0,22.0,4.0,2.0,19.0,4.0,0.0,20.0,3.0,5.0
292,17,0.936170,0.826087,0.461538,0.944444,0.750000,0.444444,1.000000,0.722222,0.636364,...,6.0,51.0,9.0,4.0,58.0,13.0,7.0,49.0,15.0,7.0
293,15,0.961538,0.863636,0.631579,0.984375,0.533333,0.800000,0.986111,0.900000,0.307692,...,12.0,63.0,8.0,8.0,71.0,18.0,4.0,63.0,17.0,9.0
294,14,0.954545,0.714286,0.666667,0.900000,0.666667,0.857143,0.937500,0.571429,0.500000,...,6.0,27.0,6.0,6.0,30.0,4.0,2.0,27.0,6.0,3.0


In [7]:
df.columns

Index(['age', 'ACC_ADD1', 'ACC_ADD2', 'ACC_ADD3', 'ACC_DIV1', 'ACC_DIV2',
       'ACC_DIV3', 'ACC_MUL1', 'ACC_MUL2', 'ACC_MUL3', 'ACC_SUB1', 'ACC_SUB2',
       'ACC_SUB3', 'RT_ADD1', 'RT_ADD2', 'RT_ADD3', 'RT_DIV1', 'RT_DIV2',
       'RT_DIV3', 'RT_MUL1', 'RT_MUL2', 'RT_MUL3', 'RT_SUB1', 'RT_SUB2',
       'RT_SUB3', 'N_ADD1', 'N_ADD2', 'N_ADD3', 'N_DIV1', 'N_DIV2', 'N_DIV3',
       'N_MUL1', 'N_MUL2', 'N_MUL3', 'N_SUB1', 'N_SUB2', 'N_SUB3'],
      dtype='object')

In [8]:
def construct_dataset(df, target_column):
    # Determine if the target is aggregated
    
    # Initialize lists to hold columns to include and exclude
    columns_to_exclude = []
    columns_to_include = []
    
    # Base columns categories
    accuracy_columns = [col for col in df.columns if col.startswith('ACC_')]
    reaction_time_columns = [col for col in df.columns if col.startswith('RT_')]
    n_of_corr_columns = [col for col in df.columns if col.startswith('N_') and 'ofCorr' not in col]

    relevant = []
    agg_name = ''
    metric = ''
    if ('ACC' in target_column) or ('Accuracy' in target_column):
        metric = 'ACC'
        relevant = accuracy_columns
        agg_name = 'Accuracy'
    elif 'RT' in target_column:
        metric = 'RT'
        relevant = reaction_time_columns
        agg_name = 'RT'
    elif ('N_' in target_column) or ('ofCorr' in target_column):
        metric = 'N' 
        relevant = n_of_corr_columns
        agg_name = 'NofCorr'


    # For non-aggregated targets, exclude related aggregated columns
    level = target_column.split('_')[-1][-1]
    operation = target_column.split('_')[-1][:-1]
    poss_col = ['ACC', 'RT', 'N']
    possible_X = []
    for l in range(1, 4):
        if l != int(level):
            columns_to_include = []
            for col in poss_col:
                columns_to_include.append(f'{col}_{operation}{l}')
            columns_to_include.append('age')
            X = df[columns_to_include]
            possible_X.append((X, f'from {l}'))

    y = df[target_column]
    
    return possible_X, y


In [9]:
X, y = construct_dataset(df, 'ACC_ADD2')

In [10]:
X[0][0].columns

Index(['ACC_ADD1', 'RT_ADD1', 'N_ADD1', 'age'], dtype='object')

In [11]:
results = {}

for name in tqdm(df.columns):
    if name == 'age':
        continue
    
    Xs, y = construct_dataset(df, name)

    for X, l in Xs:
    
        res = models.LinRegStatmodels(X, y)
        
        r2 = res.rsquared
        params = res.params.index
        
        results[f"{name} {l}"] = res

  3%|▎         | 1/37 [00:00<00:00, 89.56it/s]


MissingDataError: exog contains inf or nans

In [15]:
X.isna().sum()

ACC_ADD2    0
RT_ADD2     0
N_ADD2      2
age         0
dtype: int64

In [64]:
np.quantile(df['RT_ADD1'], 0.95)

5.7743048298

In [51]:
y

0          3.758417
1          3.985520
2          2.395548
3          8.220056
4      44958.000000
           ...     
162        1.864806
163        1.843913
164        1.986429
165        2.090514
166        2.079189
Name: RT_ADD1, Length: 167, dtype: float64

In [18]:
results['RT_DIV2'].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                RT_DIV2   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          6.691e+06
Date:                Sun, 11 Feb 2024   Prob (F-statistic):                        0.00
Time:                        17:39:30   Log-Likelihood:                         -546.35
No. Observations:                 167   AIC:                                      1121.
Df Residuals:                     153   BIC:                                      1164.
Df Model:                          14                                                  
Covariance Type:            nonrobust                                                  
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
ACC_ADD3           -7.5947      2.927     -2.594      0.010     -13.378      -1.812
ACC_MUL1          -16.9534      2.706     -6.266      0.000     -22.299     -11.608
ACC_MUL3            7.4236      2.198      3.377      0.001       3.081      11.766
RT_ADD1             0.7404      0.034     21.818      0.000       0.673       0.807
RT_ADD2            -0.7306      0.043    -16.979      0.000      -0.816      -0.646
RT_ADD3             0.7204      0.120      6.006      0.000       0.483       0.957
RT_DIV1             0.4723      0.009     52.033      0.000       0.454       0.490
RT_DIV3            -0.7188      0.119     -6.020      0.000      -0.955      -0.483
RT_MUL1            -2.0397      0.009   -224.958      0.000      -2.058      -2.022
RT_MUL2             2.4917      0.177     14.063      0.000       2.142       2.842
RT_SUB1             0.3544      0.007     52.274      0.000       0.341       0.368
RT_SUB2            -0.1179      0.002    -51.862      0.000      -0.122      -0.113
RT_SUB3            -0.1182      0.002    -52.769      0.000      -0.123      -0.114
RT ALL level1      -0.4726      0.009    -52.294      0.000      -0.490      -0.455
RT ALL ADD          0.7303      0.043     16.988      0.000       0.645       0.815
RT ALL SUB          0.1182      0.002     52.118      0.000       0.114       0.123
NofCorr ALL ADD     0.1335      0.030      4.426      0.000       0.074       0.193
==============================================================================
Omnibus:                        4.414   Durbin-Watson:                   1.864
Prob(Omnibus):                  0.110   Jarque-Bera (JB):                5.235
Skew:                          -0.141   Prob(JB):                       0.0730
Kurtosis:                       3.820   Cond. No.                     5.23e+16
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The smallest eigenvalue is 3.13e-23. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [34]:
params_frequency = {}
for name in df.columns:
    params_frequency[name] = set()

params_frequency['const'] = set()
coefs = {}
for res in results:
    for param_name in results[res].params.index:
        params_frequency[param_name].add(res)
    
    coefs[res] = results[res].params.to_dict()

In [36]:

for name in params_frequency:
    new_input = str(params_frequency[name])
    if new_input == 'set()':
        new_input = ''
    params_frequency[name] = new_input

with open("results/params_in_models_new.json", "w") as outfile:
    json.dump(params_frequency, outfile, indent=4)


r2_dict = {}
for name in results:
    r2_dict[name] = results[name].rsquared

with open("results/r2_LinReg_new.json", "w") as outfile:
    json.dump(r2_dict, outfile, indent=4)


rmse_dict = {}
for name in results:
    rmse_dict[name] = float(np.sqrt(results[name].mse_total))

with open("results/rmse_LinReg_new.json", "w") as outfile:
    json.dump(rmse_dict, outfile, indent=4)


with open("results/coefs_LinReg_new.json", "w") as outfile:
    json.dump(coefs, outfile, indent=4)

In [37]:
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor

from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

from sklearn.ensemble import RandomForestClassifier



In [38]:
from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score


In [16]:
result = {}
for name in tqdm(df.columns):
    X, y = construct_dataset(df, name)

    op, rmse, r2, model = models.XGBReg(X, y)

    result[name] = (op, rmse, r2, model)

100%|██████████| 58/58 [06:10<00:00,  6.39s/it]


In [17]:
result['ACC_ADD1']

((100, 2, 0.1, 0.7),
 0.05692233073906272,
 0.5174619389423523,
 XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, eta=0.1, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.100000001, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None))

In [17]:
rmse_dic_xgb = {}
r2_dic_xgb = {}
opt_params = {}
for elem in result:
    r2_dic_xgb[elem] = result[elem][2]
    rmse_dic_xgb[elem] = result[elem][1]
    opt_params[elem] = result[elem][0]
    

In [20]:
with open("results/rmse_XGB_new.json", "w") as outfile:
    json.dump(rmse_dic_xgb, outfile, indent=4)
with open("results/params_XGB_new.json", "w") as outfile:
    json.dump(opt_params, outfile, indent=4)
with open("results/r2_XGB_new.json", "w") as outfile:
    json.dump(r2_dic_xgb, outfile, indent=4)

RF

In [16]:
result = {}
for name in tqdm(df.columns):
    X, y = construct_dataset(df, name)

    op, rmse, r2, model = models.RFReg(X, y)

    result[name] = (op, rmse, r2, model)

100%|██████████| 58/58 [12:06<00:00, 12.53s/it]


In [18]:
rmse_dic_rf = {}
opt_params = {}
r2_dic_rf = {}

for elem in result:
    r2_dic_rf[elem] = result[elem][2]
    rmse_dic_rf[elem] = result[elem][1]
    opt_params[elem] = result[elem][0]

In [19]:
with open("results/rmse_RF_new.json", "w") as outfile:
    json.dump(rmse_dic_rf, outfile, indent=4)

with open("results/params_RF_new.json", "w") as outfile:
    json.dump(opt_params, outfile, indent=4)

with open("results/r2_RF_new.json", "w") as outfile:
    json.dump(r2_dic_rf, outfile, indent=4)

comparison


In [39]:
with open('results/rmse_RF_new.json') as file:
    res_rf = json.load(file)
with open('results/params_RF_new.json') as file:
    params_rf = json.load(file)

with open('results/rmse_XGB_new.json') as file:
    res_gb = json.load(file)

with open('results/params_XGB_new.json') as file:
    params_gb = json.load(file)

In [42]:
elem

'age'

In [45]:
FI_RF = {}
for elem in tqdm(res_rf):
    opt_params = params_rf[elem]
    model = RandomForestRegressor(n_estimators=opt_params[0], max_depth=opt_params[1], min_samples_leaf=opt_params[2], min_samples_split=opt_params[3], n_jobs=-1)
    X, y = construct_dataset(df, elem)
    model.fit(X,y)
    FI = list(zip(model.feature_importances_, X.columns))
    FI_RF[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]])


  0%|          | 0/58 [00:00<?, ?it/s]

  5%|▌         | 3/58 [00:01<00:27,  1.97it/s]


AttributeError: 'RandomForestRegressor' object has no attribute 'get_booster'

In [46]:

FI_GB = {}
for elem in tqdm(res_rf):
    opt_params = params_gb[elem]
    model = XGBRegressor(n_estimators = opt_params[0], max_depth=opt_params[1], eta=opt_params[2], colsample_bytree=opt_params[3])

    X, y = construct_dataset(df, elem)
    model.fit(X,y)

    FI = list(zip(model.feature_importances_, model.get_booster().feature_names))
    FI_GB[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]])

# with open('FeatureImportance.json', 'w') as file:
#     json.dump(FI5, file, indent=4)

100%|██████████| 58/58 [00:06<00:00,  9.61it/s]


In [48]:
with open('results/FI_RF.json', 'w') as file:
    json.dump(FI_RF, file, indent=4)

In [50]:
with open('results/FI_GB.json', 'w') as file:
    json.dump(FI_GB, file, indent=4)

In [43]:
opt_models = {}
for elem in tqdm(res_rf):
    diff = res_rf[elem] - res_gb[elem]
    if diff > 0:
        opt_params = params_gb[elem]
        model = XGBRegressor(n_estimators = opt_params[0], max_depth=opt_params[1], eta=opt_params[2], colsample_bytree=opt_params[3])
    else:
        opt_params = params_rf[elem]
        model = RandomForestRegressor(n_estimators=opt_params[0], max_depth=opt_params[1], min_samples_leaf=opt_params[2], min_samples_split=opt_params[3], n_jobs=-1)

    X, y = construct_dataset(df, elem)
    model.fit(X,y)

    opt_models[elem] = model  


FI5 = {}
for elem in opt_models:
    if isinstance(opt_models[elem], XGBRegressor):
        FI = list(zip(opt_models[elem].feature_importances_, opt_models[elem].get_booster().feature_names))
        FI5[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]], 'GB')
    else:
        X, y = construct_dataset(df, elem)
        FI = list(zip(opt_models[elem].feature_importances_, X.columns))
        FI5[elem] = ([(elem[1], float(elem[0])) for elem in sorted(FI, key = lambda x: x[0], reverse=True)[:5]], 'RF')

    
# with open('FeatureImportance.json', 'w') as file:
#     json.dump(FI5, file, indent=4)

  0%|          | 0/58 [00:00<?, ?it/s]

 50%|█████     | 29/58 [00:16<00:16,  1.81it/s]


KeyboardInterrupt: 

In [41]:
with open('FeatureImportance5_new.json', 'w') as file:
     json.dump(FI5, file, indent=4)